In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('Tulu_NER - Tulu_NER.csv')

In [3]:
data.head(5)

,Words,NER_Tag
0,ದಕ್ಷಿಣ,B-LOC
1,ಕನ್ನಡ,I-LOC
2,ಜಿಲ್ಲೆಗ್,B-GPE
3,ತುರ್ತು,B-MED
4,ಆಕ್ಸಿಜನ್,B-MED


In [4]:
from textblob import TextBlob
#import emot
import nltk.data
import numpy as np
import re
import time
import string
#import emoji
#import demoji
#import contractions
import nltk
from nltk.corpus import stopwords

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['NER_Tag'])
#y_test =label_encoder.transform(y_test)

In [6]:
# y_test

In [7]:
from tqdm import tqdm #dont run this
benvocab = []
embedding_index_kan = {}
f1 = open('C:/Users/CCL/Downloads/cc.kn.300 (1).vec/cc.kn.300.vec',encoding='utf-8')
for line in f1:
        values = line.rstrip().split(' ')
        word = values[0]
        benvocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_kan[word] = coefs

In [8]:
from tqdm import tqdm #dont run this
benvocab1 = []
embedding_index_tul = {}
f2 = open('wiki.tcy (2).vec',encoding='utf-8')
for line in f2:
        values = line.rstrip().split(' ')
        word = values[0]
        benvocab1.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_tul[word] = coefs

In [9]:
embedding_index_kan.update(embedding_index_tul)

In [10]:
import pickle #dont run this
with open('embedding_index_kan.pkl', 'wb') as f:  # open a text file
    pickle.dump(embedding_index_kan, f) # serialize the list

In [11]:
def get_vectors(sentence):# dont run
    sentvec = np.zeros(300)
    zeros = np.zeros(300)

    for word in sentence.split():
        if word in benvocab:
             temp = embedding_index_kan[word]
        else: temp = zeros
        # print('word : ', word, ' ttt : ', temp)
        sentvec += temp

    sentvec = sentvec / len(sentence.split())
    return sentvec

In [12]:
x=data['Words'].tolist()

In [13]:
X=pd.DataFrame(x)
Y=pd.DataFrame(y)

In [14]:
X=X.rename(columns={0:'Words'})

In [15]:
Y=Y.rename(columns={0:'NER_Tag'})

In [16]:
df=pd.concat([X,Y], axis=1)

In [17]:
df['text']=df['Words'].astype(str)

In [18]:
df['vec'] = df['text'].apply(lambda x:get_vectors(x))#dont run this

In [20]:
import pickle #dont run this
with open('vectrain_file1.pkl', 'wb') as f:  # open a text file
    pickle.dump(df['vec'], f) # serialize the list

In [22]:
import pickle
with open('vectrain_file1.pkl', 'rb') as f:

    train_loaded = pickle.load(f) # deserialize using load()

In [23]:
df=df.rename(columns={'\tNER_Tag': 'NER_Tag'})

In [24]:
df

,Words,NER_Tag,text,vec
0,ದಕ್ಷಿಣ,9,ದಕ್ಷಿಣ,"[0.10593000054359436, -0.07414799928665161, -0..."
1,ಕನ್ನಡ,27,ಕನ್ನಡ,"[-0.1512800008058548, -0.008274099789559841, -..."
2,ಜಿಲ್ಲೆಗ್,6,ಜಿಲ್ಲೆಗ್,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ತುರ್ತು,10,ತುರ್ತು,"[-0.008799999952316284, -0.052799999713897705,..."
4,ಆಕ್ಸಿಜನ್,10,ಆಕ್ಸಿಜನ್,"[0.003599999938160181, 0.011900000274181366, -..."
...,...,...,...,...
20202,ದಕ್ಷಿಣ,9,ದಕ್ಷಿಣ,"[0.10593000054359436, -0.07414799928665161, -0..."
20203,ಕನ್ನಡ,27,ಕನ್ನಡ,"[-0.1512800008058548, -0.008274099789559841, -..."
20204,ಜಿಲ್ಲಾಡಳಿತಗ್,6,ಜಿಲ್ಲಾಡಳಿತಗ್,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
20205,ಕೊರ್ತೆರ್,36,ಕೊರ್ತೆರ್,"[-0.2126999944448471, -0.44909000396728516, -0..."


In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train_loaded, df.NER_Tag, test_size=0.3)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [27]:
MNB = GaussianNB()
MNB.fit(X_train.tolist(), y_train.tolist())
y_pred_MNB= MNB.predict(X_test.tolist())
print( classification_report(y_test, y_pred_MNB))

              precision    recall  f1-score   support

           0       0.26      0.21      0.24       233
           1       0.85      0.45      0.59        38
           2       0.39      0.46      0.42       208
           3       0.00      0.00      0.00        23
           4       0.20      0.27      0.23        95
           5       0.21      0.10      0.14        69
           6       0.28      0.42      0.33        77
           7       0.00      0.00      0.00         9
           8       0.03      0.01      0.02        77
           9       0.75      0.14      0.23       243
          10       0.18      0.26      0.22       183
          11       0.77      0.54      0.63       164
          12       0.12      0.17      0.14       210
          13       0.40      0.11      0.17       188
          14       0.67      0.05      0.09       170
          15       0.00      0.00      0.00       213
          16       0.10      0.50      0.17         2
          17       0.00    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classificat

In [28]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train.tolist(), y_train.tolist())
y_pred_svm = svm_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.45      0.14      0.21       233
           1       0.76      0.34      0.47        38
           2       0.76      0.56      0.65       208
           3       0.00      0.00      0.00        23
           4       0.89      0.18      0.30        95
           5       0.74      0.20      0.32        69
           6       0.47      0.26      0.33        77
           7       0.00      0.00      0.00         9
           8       0.71      0.06      0.12        77
           9       0.64      0.44      0.52       243
          10       0.57      0.25      0.35       183
          11       0.96      0.65      0.78       164
          12       0.24      0.15      0.18       210
          13       0.54      0.50      0.52       188
          14       0.68      0.19      0.29       170
          15       0.53      0.28      0.36       213
          16       0.00      0.00      0.00         2
          17       0.57    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
from sklearn.svm import LinearSVC
lsvc_model = LinearSVC(C=1.0, class_weight='balanced')
lsvc_model.fit(X_train.tolist(), y_train.tolist())
y_pred_lsvc = lsvc_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_lsvc))

              precision    recall  f1-score   support

           0       0.43      0.21      0.28       233
           1       0.57      0.53      0.55        38
           2       0.58      0.63      0.60       208
           3       0.18      0.35      0.24        23
           4       0.42      0.31      0.35        95
           5       0.36      0.29      0.32        69
           6       0.55      0.52      0.53        77
           7       0.14      0.33      0.19         9
           8       0.25      0.18      0.21        77
           9       0.60      0.44      0.51       243
          10       0.59      0.34      0.44       183
          11       0.95      0.65      0.77       164
          12       0.26      0.17      0.20       210
          13       0.55      0.53      0.54       188
          14       0.42      0.21      0.28       170
          15       0.62      0.22      0.33       213
          16       0.07      1.00      0.14         2
          17       0.30    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [30]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train.tolist(), y_train.tolist())
y_pred_lr = lr_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.46      0.18      0.25       233
           1       0.73      0.29      0.42        38
           2       0.79      0.53      0.63       208
           3       0.00      0.00      0.00        23
           4       0.82      0.19      0.31        95
           5       0.67      0.14      0.24        69
           6       0.50      0.34      0.40        77
           7       0.00      0.00      0.00         9
           8       0.38      0.04      0.07        77
           9       0.68      0.39      0.49       243
          10       0.51      0.22      0.31       183
          11       0.95      0.65      0.77       164
          12       0.24      0.12      0.16       210
          13       0.55      0.50      0.53       188
          14       0.60      0.18      0.27       170
          15       0.64      0.24      0.35       213
          16       0.00      0.00      0.00         2
          17       0.50    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels 

In [31]:
RF_model = RandomForestClassifier()
RF_model.fit(X_train.tolist(), y_train.tolist())
y_pred_rf = RF_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.59      0.30      0.40       233
           1       0.81      0.55      0.66        38
           2       0.87      0.69      0.77       208
           3       0.62      0.43      0.51        23
           4       0.84      0.38      0.52        95
           5       0.77      0.33      0.46        69
           6       0.73      0.47      0.57        77
           7       0.75      0.33      0.46         9
           8       0.66      0.25      0.36        77
           9       0.77      0.43      0.55       243
          10       0.71      0.38      0.50       183
          11       0.95      0.63      0.76       164
          12       0.46      0.33      0.38       210
          13       0.54      0.59      0.56       188
          14       0.71      0.41      0.52       170
          15       0.71      0.31      0.43       213
          16       1.00      1.00      1.00         2
          17       0.75    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train.tolist(), y_train.tolist())
y_pred_xgb = xgb_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.53      0.30      0.39       233
           1       0.79      0.50      0.61        38
           2       0.87      0.68      0.76       208
           3       0.59      0.43      0.50        23
           4       0.76      0.37      0.50        95
           5       0.69      0.35      0.46        69
           6       0.71      0.47      0.56        77
           7       0.67      0.22      0.33         9
           8       0.54      0.25      0.34        77
           9       0.72      0.42      0.53       243
          10       0.67      0.38      0.48       183
          11       0.94      0.65      0.77       164
          12       0.45      0.34      0.39       210
          13       0.59      0.52      0.55       188
          14       0.62      0.41      0.49       170
          15       0.66      0.31      0.42       213
          16       1.00      1.00      1.00         2
          17       0.75    

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
